In [1]:
%cd ..

/Users/juansegundohevia/Documents/repos/TexTract


In [55]:
from torchtext.data import bleu_score
import pickle as pkl
import os
from transformers import PreTrainedTokenizerFast
import numpy as np
import pix2tex.cli as p2t


def detokenize(tokens, tokenizer):
    toks = [tokenizer.convert_ids_to_tokens(tok) for tok in tokens]
    for b in range(len(toks)):
        for i in reversed(range(len(toks[b]))):
            if toks[b][i] is None:
                toks[b][i] = ''
            toks[b][i] = toks[b][i].replace('Ġ', ' ').strip()
            if toks[b][i] in (['[BOS]', '[EOS]', '[PAD]']):
                del toks[b][i]
    return toks

def get_tokens(prediction, label):

    tokenizer = PreTrainedTokenizerFast(tokenizer_file="model/dataset/tokenizer.json")
    pred = detokenize(prediction, tokenizer)
    truth = detokenize(label, tokenizer)
    
    return pred, truth
#bleus.append(metrics.bleu_score(pred, [alternatives(x) for x in truth]))

# HW data predictions

In [3]:
with open("notebooks/preds/hw_preds.pkl", "rb") as f:
    hw_preds = pkl.load(f)

with open("notebooks/preds/original_preds.pkl", "rb") as f:
    original_preds = pkl.load(f)

In [4]:
# get hw labels
with open("pix2tex/dataset/handwritten/CROHME_math.txt", "r") as f:
    hw_labels = f.readlines()

hw_labels[:2]

['y = A x + A ^ { 2 }\n',
 'B _ { n } ( 1 - x ) = ( - 1 ) ^ { n } B _ { n } ( x )\n']

In [42]:
from nltk.translate.bleu_score import sentence_bleu
reference = [["this", "is", "a", "test"], ["this", "is", "test"]]
candidate = ["this", "is", "a", "test"]
score = sentence_bleu(reference, candidate)
print(score)

1.0


# Try Eval

In [58]:
from pix2tex.dataset.dataset import Im2LatexDataset

with open("pix2tex/dataset/handwritten/test_hw.pkl", "rb") as f:
    test_hw = pkl.load(f)

In [60]:
from pix2tex.eval import evaluate

In [80]:
from PIL import Image
import os
import numpy as np
import pix2tex as p2t
from munch import Munch
import yaml
from pix2tex import cli as p2t
import matplotlib.pyplot as plt
from pix2tex.dataset.dataset import Im2LatexDataset
from pix2tex.eval import evaluate
from torchtext.data import metrics

hw_config = Munch({
    "config": "settings/handwritten_training.yaml",
    "checkpoint": "../../hw_checkpoints/handwritten_training/handwritten_training_e19_step63.pth",
    "no_cuda": True,
    "no_resize": False
})

original_config = Munch({
    "config": "settings/config.yaml",
    "checkpoint": "model/checkpoints/weights.pth",
    "no_cuda": True,
    "no_resize": False
}) 
hw_model = p2t.LatexOCR(hw_config)
original_model = p2t.LatexOCR()
# load test set for handwritten files
hw_test_set = Im2LatexDataset().load("pix2tex/dataset/handwritten/test_v2.pkl")
original_test_set = Im2LatexDataset().load("pix2tex/dataset/formulae/test.pkl")
# load yaml files to parse configurations
with open("pix2tex/model/settings/handwritten_training.yaml", 'r') as f:
    hw_config_yaml = Munch(yaml.safe_load(f))

with open("pix2tex/model/settings/config.yaml", 'r') as f:
    original_config_yaml = Munch(yaml.safe_load(f))

hw_config_yaml.device = "cpu"

In [78]:
hw_config_yaml.pad

True

In [93]:
from pix2tex import eval
import importlib
importlib.reload(eval)

<module 'pix2tex.eval' from '/Users/juansegundohevia/Documents/repos/TexTract/pix2tex/eval.py'>

In [94]:
bleu_score, edit_distance, token_accuracy, failure_count, bleus, edit_dists, token_acc = eval.evaluate_everything(hw_model.model, hw_test_set, hw_config_yaml)

100%|██████████| 1083/1083 [12:57<00:00,  1.39it/s]


UnboundLocalError: local variable 'truth' referenced before assignment